In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
trade_df= pd.read_csv("BTC-USD.csv")

In [ ]:
trade_df.head()

In [ ]:
trade_df.info()

In [ ]:
trade_df=trade_df.drop(columns=['Open','High','Low','Close','Volume'])

## This is just one column ..so we will need it split in 2 columns by skipping row 1

In [ ]:
trade_df.rename(columns={'Date':'Week'}, 
                 inplace=True)

In [ ]:
trade_df.head()

In [ ]:
trade_df.plot()

In [ ]:
from datetime import datetime
trade_df['Week']=  pd.to_datetime(trade_df['Week']).dt.strftime('%d-%m-%Y')

In [ ]:
trade_df['Week']=[str(i) for i in trade_df['Week']]

In [ ]:
trade_df

In [ ]:
trade_df['Week']=pd.to_datetime(trade_df['Week'])

In [ ]:
trade_df['Week']

In [ ]:
trade_df.set_index('Week', inplace=True)

In [ ]:
trade_df.info()

In [ ]:
trade_df.index

In [ ]:
trade_df['01-2018']

In [ ]:
trade_df['2018']

In [ ]:
trade_df.plot()

In [ ]:
trade=trade_df['Adj Close']
# above is a series if data ..

In [ ]:
trade.head()

In [ ]:
type(trade)

In [ ]:
type(trade_df)

# model ARIMA(p,d,q)

# decomposing time series data 
1. seasonality
2. peak in some time of year
3. repetative beahiviur
4. residual: nosie
    

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition=seasonal_decompose(trade, model='multiplicative', freq=52)

In [ ]:
decomposition.plot()

In [ ]:
trend= decomposition.trend
seasonal=decomposition.seasonal
residual=decomposition.resid

    

In [ ]:
plt.figure(figsize=(10,10))
plt.figure(1)
plt.subplot(411)
plt.plot(trend*seasonal*residual, label='Original')
plt.plot(trend,label='Trend',linestyle='--')
plt.legend(loc='best')


plt.subplot(412)

plt.plot(seasonal,label='seasonal' )
plt.legend(loc='best')

plt.subplot(413)

plt.plot(residual,label='Residuals')
plt.legend(loc='best')




# (Partial) Autocorrelation Function

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
plot_acf(trade)
plt.xlim(0,50)

In [ ]:
diff = trade.diff(1).dropna()

In [ ]:
diff.plot()

In [ ]:

decomposition=seasonal_decompose(diff, model='additive', freq=52)



In [ ]:
decomposition.plot()

# d = 1

In [ ]:
plot_acf(diff)
plt.xlim(0,20)

In [ ]:
# for q :  pick a number where in disapper in blue area.. here it is first record.. so will pick the value of 2

###### q=3

# looking for p variable

In [ ]:
plot_pacf(diff, lags=10)

# for p pick a number where in disapper in blue area.. here it is first record.. so will pick the value of 4
# p=3

In [ ]:
# p=4,  d=1, q=3,

# Building ARIMA model

In [ ]:
# for splitting data we need to split by time rather than randomly picking a data set for test and train~

In [ ]:
train = trade[:'06-2018'].dropna()
test = trade['07-2018':].dropna()


In [ ]:
train.tail()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model= SARIMAX(train, order=(3,1,3), seasonal_order=(0,0,0,0), enforce_stationarity=False, enforce_invertibility=False)

In [ ]:
results = model.fit()

In [ ]:
results.plot_diagnostics()

In [ ]:
pred = results.predict(start=200,end=250)

In [ ]:
pred.head()

In [ ]:
results.summary()

In [ ]:
# comparing to training set

In [ ]:
fit= results.predict(start='06-2014', end='06-2018')[1:]

plt.figure(figsize=(15,8))
plt.plot(train, label='Train')
plt.plot(fit, label='Fit')
#plt.plot(train, label='Train')
plt.legend(loc='best')
